In [ ]:
import re

def clean_text_red(a):
    return re.sub(' ','', a)

def clean_text(a):
    return re.sub('[^0-9]','', a)

def analyze_data(results, date_regex):
    for (bbox, text, prob) in results:
        clean_text = clean_text_red(text)
        print(clean_text)
        extract = re.search(date_regex, clean_text)
        if(extract is not None):
            return extract.group()  
                    
    merged = merge_intervals(results)

    for (bbox, text, prob) in merged:
        clean_text = clean_text_red(text)
        extract = re.search(date_regex, clean_text)
        if(extract is not None):
            return extract.group()  
        
    return "not found"

def merge_intervals(results):
    #results.sort(key=lambda x : x[0][0][0])
    
    results = sorted(results, key=lambda x : x[0][0][0])
    for i in range(0, len(results)):
        bbox = results[i][0]
        (tl, tr, br, bl) = bbox

        for k in range(i + 1, len(results)):
            bbox_prev = results[k][0]
            (tl2, tr2, br2, bl2) = bbox_prev
            # if(results[k][1] == "MAR"):
            #print(results[i][1] + "  " + results[k][1]  + "   " + str(abs(tr2[0] - tl[0])) + "  " + str(abs(tr2[1] - tl[1])))

            if(abs(tl2[0] - tr[0]) < 45 and abs(tl2[1] - tr[1]) < 45):
                #print(results[i][1] + "    Prev: " + results[k][1])
                new_string = results[i][1] + results[k][1]
                results[i] = (results[i][0], new_string, results[i][2])
                results[k] = (results[k][0], new_string, results[k][2])

    print("END")
    return results
    

In [ ]:
import pandas as pd
import PathStorage
import re
from easyocr import Reader
import argparse
import cv2
import PathStorage
import os
import DateConversion
import time
from PIL import Image, ImageOps
import pymupdf
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import Testing.pdfHelper as ph
import scipy.ndimage as nd
import matplotlib.patches as patches

image_temp_path = r"C:\Users\PaGarcia\Pictures\temp.png"

red_threshold = 1000000 #measured = 2209830.0

month_alpha = "janfebmrpyulgsoctvd0123456789 "
month_alpha = month_alpha.upper()
num_alpha = "0123456789 "

def split(array, start, end):
    start_pos = int(array.shape[0] * start)
    end_pos = int(array.shape[0] * end)
    return array[start_pos:end_pos,:,:]

def red(new):
    new = new.astype(np.float32)
    new[:,:,0] -= new[:,:,1] * .52
    new[:,:,0] -= new[:,:,2] * .52
    new[:,:,1] = 0
    new[:,:,2] = 0
    new[:,:,0] *= 100000
    new[:,:,0] = np.clip(new[:,:,0], 0, 255)
    new[:,:,1] = new[:,:, 0]
    new[:,:,2] = new[:,:, 0]
    new = nd.minimum_filter(new, 2)
    return new
    
def compute_red_sum(new):
    new = new.astype(np.float32)
    new[:,:,0] -= new[:,:,1] * .8
    new[:,:,0] -= new[:,:,2] * .8
    new[:,:,0] *= 100000
    new[:,:,0] = np.clip(new[:,:,0], 0, 255)
    return np.sum(new[:,:,0])

def isolate_grey(array):
    array[array < 90] = 255
    array[array < 200] = 0
    array = nd.median_filter(array, 3)
    #array = nd.maximum_filter(array, 3)
    array[array < 220] = 0
    array = nd.maximum_filter(array, 3)
    array[array > 1] = 255
    array = nd.gaussian_filter(array, 3)
    array = nd.maximum_filter(array, 3)
    array[array < 200] = 0
    array[array > 1] = 255
    new = array.astype(np.int32)
    return new

doc_data = pd.read_csv(PathStorage.docdata_sheet, index_col=0)
doc_data = doc_data.astype({"Date" : "object"})
temp = doc_data.fillna("")

pdf_path = doc_data.loc[13, "Path"]
#pdf_path = r"C:\Users\PaGarcia\Documents\NewWorkflow\Final\DCR_15523_RE_20180719_Renewal-DS.pdf"

doc = pymupdf.open(pdf_path)
page = doc.load_page(0)
pix = page.get_pixmap(dpi=300)

image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
array = np.asarray(image, dtype=np.int32)

bottom_slice = split(array, .85, 1)

red_sum = compute_red_sum(np.copy(bottom_slice))

is_red = red_sum > red_threshold

print(str(is_red) + " " + str(red_sum))

if(is_red):
    red_copy = red(np.copy(bottom_slice))
    im = Image.fromarray(red_copy.astype(np.uint8))
    alphabet = month_alpha
else:
    top_slice = split(array, 0, .125)
    grey_copy = isolate_grey(np.copy(top_slice))
    im = Image.fromarray((grey_copy).astype(np.uint8))
    alphabet = num_alpha


im.save(r"C:\Users\PaGarcia\Pictures\redPath2.png")
cv2_version = cv2.imread(r"C:\Users\PaGarcia\Pictures\redPath2.png")

reader = Reader(["en"], False)
results = reader.readtext(cv2_version, allowlist=alphabet)
#array = Image.fromarray(new, 'RGB')
#img
full_grey_date = r"((20)([0-1][0-9]|2[0-4]))(0[1-9]|1[0,1,2])(0[1-9]|[12][0-9]|3[01])"
full_red_date = r"([a-zA-Z]{3})(0[1-9]|[12][0-9]|3[01])((20)([0-1][0-9]|2[0-4]))"

regex =  full_red_date if is_red else full_grey_date
prediction = analyze_data(results, regex)
print("prediction:  " + prediction)

fig, ax = plt.subplots()
ax.imshow(cv2_version, interpolation=None)

#print(results)

# loop over the results
for (bbox, text, prob) in results:
	# display the OCR'd text and associated probability
	#print("[INFO] {:.4f}: {}".format(prob, text))
	# unpack the bounding box
	(tl, tr, br, bl) = bbox
	tl = (int(tl[0]), int(tl[1]))
	tr = (int(tr[0]), int(tr[1]))
	br = (int(br[0]), int(br[1]))
	bl = (int(bl[0]), int(bl[1]))
	# cleanup the text and draw the box surrounding the text along
	# with the OCR'd text itself
	#text = cleanup_text(text)

	font = { 'size'   : 4,
            'color' : "#FFFFFF"}
	rect = patches.Rectangle(bl, tr[0] - bl[0], tr[1] - bl[1], linewidth=1, edgecolor='r', facecolor='none')
	plt.text(tl[0], tl[1], text, **font)

	ax.add_patch(rect)

#results

In [ ]:
from difflib import SequenceMatcher

print(SequenceMatcher(None, "Jan", "Jap").ratio())
results